In [ ]:
import os
import sagemaker
import numpy as np
import tensorflow as tf
from sagemaker.tensorflow import TensorFlow

In [ ]:
ON_SAGEMAKER_NOTEBOOK = True

# train_instance_type='local_gpu'
train_instance_type='ml.p2.xlarge'
deploy_instance_type='ml.c5.2xlarge'

bucket = 'quikr-datascience-mumbai'
key = 'image_classify/ena_classify/data'

In [ ]:
train_input_path = "s3://{}/{}/train/".format(bucket, key)
validation_input_path = "s3://{}/{}/validation/".format(bucket, key)

In [ ]:
sagemaker_session = sagemaker.Session()
if ON_SAGEMAKER_NOTEBOOK:
    role = sagemaker.get_execution_role()
else:
    role = '[IAM ROLE]'

In [ ]:
hyperparameters = {'epochs': 100, 'batch_size' : 64}

In [ ]:
os.path.abspath('')

In [ ]:
estimator = TensorFlow(base_job_name='ena-classify-mobilenet-2',
#                        entry_point=os.path.join(os.path.abspath(''),'entry.py'),
                       entry_point='entry.py',
                       source_dir='src',
                       model_dir='s3://quikr-datascience-mumbai/image_classify/ena_classify/sagemaker_output',
                       code_location='s3://quikr-datascience-mumbai/image_classify/ena_classify/sagemaker_output',
                       role=role,
                       framework_version='2.0.0',
                       py_version='py3',
                       output_path='s3://quikr-datascience-mumbai/image_classify/ena_classify/sagemaker_output',
                       hyperparameters=hyperparameters,
                       train_instance_count=1, train_instance_type=train_instance_type,
                       tags=[{'Key':'project', 'Value':'image-classify-102'}])

In [ ]:
print("Training ...")
local_inputs = {'train' : train_input_path, 
                'validation' : validation_input_path}
estimator.fit(inputs=local_inputs)

## Deploying

In [ ]:
from sagemaker.tensorflow.serving import Model

In [ ]:
!ls

In [ ]:
model = Model(entry_point='inference.py',
#               dependencies=['requirements.txt'],
              source_dir='./src',
              model_data='s3://quikr-datascience-mumbai/image_classify/ena_classify/sagemaker_output/ena-classify-mobilenet-2-2020-02-20-21-31-56-215/output/model.tar.gz',
              framework_version='2.0.0', role=role)
predictor = model.deploy(initial_instance_count=1, endpoint_name='quikr-eye-v1', instance_type=deploy_instance_type,
                        tags=[{'Key':'project', 'Value':'image-classify-102'}])